In [ ]:
import h3
import geopandas
import geodatasets
import contextily as cx
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df = geopandas.read_file(geodatasets.get_path('nybb'))
df = df.to_crs(epsg=4326)
df.crs

In [ ]:
df

In [ ]:
a = df.geometry.apply(lambda x: h3.geo_to_cells(x, res=8)).apply(h3.cells_to_h3shape)

In [ ]:
a

In [ ]:
a[2]

```
<H3MultiPoly: <178/(6,)>, <66/(6,)>, <22>, <30>, <10>, <6>, <6>>
```

In [ ]:
type(df.geometry[0])

In [ ]:
def foo(df):
    df = df.to_crs(epsg=3857)
    
    ax = df.plot(figsize=(10, 10), alpha=0.5, edgecolor='k', column='BoroName', categorical=True, legend=True, legend_kwds={'loc': 'upper left'})
    
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    cx.add_basemap(ax, crs=df.crs, source=cx.providers.CartoDB.Positron)

In [ ]:
foo(df)

In [ ]:
dfh3 = df.copy()

dfh3.geometry = dfh3.geometry.apply(lambda x: h3.geo_to_cells(x, res=9)).apply(h3.cells_to_h3shape)

In [ ]:
foo(dfh3)

In [ ]:
import geodatasets as gds
import geopandas as gpd
import geoviews as gv
import cartopy.crs as ccrs

gv.extension('bokeh');

In [ ]:
def bar(df, epsg):
    crs = ccrs.epsg(epsg)
    df = df.to_crs(crs)
    
    tiles = gv.tile_sources.CartoLight
    polys = gv.Polygons(df, vdims=['BoroName'], crs=df.crs)

    out = tiles * polys.opts(color='BoroName', cmap='Pastel1', tools=['hover'], width=1200, height=800, alpha=0.5, active_tools=['wheel_zoom'], xaxis=None, yaxis=None)

    return out

In [ ]:
bar(df, 3857)

# converting to cells

In [ ]:
df = geopandas.read_file(geodatasets.get_path('nybb'))
df = df[['BoroName', 'geometry']]
df = df.to_crs(epsg=4326)
df

In [ ]:
df['boro_geo'] = df.BoroName

In [ ]:
res = 9

df1 = df.copy()
df2 = df.copy()

df1['boro_geo'] = df1.BoroName

df2.geometry = df2.geometry.apply(lambda x: h3.geo_to_cells(x, res=res)).apply(h3.cells_to_h3shape)
df2['boro_geo'] = df2.BoroName + ' H3'
df3 = pd.concat([df1, df2])
df3

In [ ]:
def bar(df, epsg):
    crs = ccrs.epsg(epsg)
    df = df.to_crs(crs)
    
    tiles = gv.tile_sources.CartoLight
    polys = gv.Polygons(df, crs=df.crs, vdims=['BoroName', 'boro_geo'])

    out = tiles * polys.opts(color='BoroName', cmap='Pastel1', tools=['hover'], width=1200, height=800, alpha=0.5, active_tools=['wheel_zoom'], xaxis=None, yaxis=None)

    return out

In [ ]:
epsg = 3857
crs = ccrs.epsg(epsg)
df = df.to_crs(crs)
polys = gv.Polygons(df, crs=df.crs, vdims=['BoroName', 'boro_geo'])

In [ ]:
polys?

In [ ]:
bar(df3, 3857)

In [ ]:
df